In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# set up arrays
dx = 1000
x = np.arange(0, 100000, dx)
x_node = x + (dx/2)

Hstar = 0.5
etab = np.zeros(len(x)) # bedrock elevation, m
etab[:] = np.linspace(10, 0.1, len(x)) # need to start with slight bedrock slope
H = Hstar + np.zeros(len(x))
eta = etab + H

In [3]:
# set up parameters
# K_ig = np.zeros(len(x))
# K_ig[:24] = 1e-7
# K_sed = np.zeros(len(x))
# K_sed[24:] = 1e-6
K = np.zeros(len(x))
K[:] = 1e-7

# beta_ig = np.zeros(len(x))
# beta_ig[:24] = 4e-7
# beta_sed = np.zeros(len(x))
# beta_sed[24:] = 4e-6
beta = np.zeros(len(x))
beta[:] = 1e-5

# psi = np.zeros(len(x))
# psi[:14] = 4e-7
# psi[14:] = 4e-6
psi = 4e-5
gamma = 0.3

r = 1. # runoff rate
D = 0.015 # grain size
phi = 0.45 # porosity
baselevel_rate = 0.0001 # baselevel fall rate

In [4]:
kqs = 0.041
I = 0.01
kxb = 25
Pxb = 1./5.
kb = 8.3e-8

B_valley = kxb * (x_node**Pxb)
B_basin = (2./3.) * x_node

area = (1./3.) * x**2
# Q = r * x * B
Q = r * area

In [5]:
def model(x, dx, x_node,
          K, beta,
          D, phi, Hstar, psi, gamma,
          kb, kqs, I, r, B_valley, B_basin, Q,
          etab, H, eta,
          baselevel_rate,
          num_steps = 1):
    
    q = Q/B_valley
    # dt = 0.2 * dx * dx / (kqs * q[-1]) # try changing to 10, 5, or 1 if this is unstable
    dt = 5
    
    run_duration = dt * num_steps
    cum_time = 0.0
    
    Qs = np.zeros(len(x))
    qs = np.zeros(len(x))
    Eb_p = np.zeros(len(x))
    Eb_a = np.zeros(len(x))
    Eb = np.zeros(len(x))
    Eh = np.zeros(len(x))
    E = np.zeros(len(x))
    ql = np.zeros(len(x))
    
    H[-1] = 0
    E[-1] = baselevel_rate
    q[0] = 0
    Qs[0] = 0
    ql[0] = 0
    
    while cum_time < run_duration:
        
        # drop baselevel
        eta[-1] -= baselevel_rate * dt
        
        # set boundary conditions
        etab[-1] = eta[-1]
        
        # calc slope
        S = -np.diff(eta)/dx
        
        # calc bed exposure
        alpha = np.exp(-H/Hstar)
        
        # calc Rh
#         width = (kb * Q[1:] * (S ** (7/6))) / (D**(3/2))
#         depth = (0.09801 * D) / S
#         Rh_efficiency = 1 - np.exp(-width/(2*depth))
        
        # calc sed transport
        Qs[1:] = kqs * I * Q[1:] * S**(7./6.) * (1 - alpha[:-1]) # * Rh_efficiency
        qs[1:] = Qs[1:] / B_valley[1:]
        
        # calc bedrock erosion
        Eb_p[:-1] = K[1:] * Q[1:] * S * alpha[:-1]
        Eb_a[:-1] = beta[1:] * Qs[1:] * alpha[:-1]
        Eb[:-1] = (Eb_p[:-1] + Eb_a[:-1]) / B_valley[:-1]
        
        # calc grain attrition
        atr = Qs * psi
        
        # calc equilibrium lateral supply
        ql[1:] = ((baselevel_rate * gamma) / psi) * (1 - np.exp(-psi/(3 * x[1:])))
        # ql[24:] = 0
        
        # calc sediment erosion rate
        Eh[:-1] = 1/((1 - phi) * B_valley[:-1]) * (np.diff(Qs)/dx + atr[1:] - Eb_p[:-1] - ql[1:])
        
        # calc total erosion rate
        E[:-1] = Eb[:-1] + Eh[:-1]
        
        # update topography
        etab[:-1] -= Eb[:-1] * dt
        H[:-1] -= Eh[:-1] * dt
        H[H<0] = 0
        eta[:-1] = etab[:-1] + H[:-1]
        
        # update elapsed time
        cum_time += dt
        
    print(cum_time)
        
    return (dt, S, q, alpha, Qs, qs, Eb_p, Eb_a, Eb, atr, ql, Eh, E, etab, H, eta)

In [6]:
#now write a test
num_steps = 10000000
test_run = [x, dx, x_node, 
            K, beta, 
            D, phi, Hstar, psi, gamma,
            kb, kqs, I, r, B_valley, B_basin, Q,
            etab, H, eta,
            baselevel_rate,
            num_steps]

# and run it
(dt, S, q, alpha, Qs, qs, Eb_p, Eb_a, Eb, atr, ql, Eh, E, etab, H, eta) = model(*test_run)

50000000.0


In [7]:
E

array([9.99939666e-05, 9.99984412e-05, 9.99990725e-05, 9.99993433e-05,
       9.99994976e-05, 9.99995983e-05, 9.99996695e-05, 9.99997224e-05,
       9.99997633e-05, 9.99997958e-05, 9.99998221e-05, 9.99998438e-05,
       9.99998619e-05, 9.99998773e-05, 9.99998904e-05, 9.99999017e-05,
       9.99999116e-05, 9.99999201e-05, 9.99999277e-05, 9.99999343e-05,
       9.99999402e-05, 9.99999455e-05, 9.99999501e-05, 9.99999544e-05,
       9.99999581e-05, 9.99999615e-05, 9.99999647e-05, 9.99999674e-05,
       9.99999699e-05, 9.99999723e-05, 9.99999743e-05, 9.99999763e-05,
       9.99999780e-05, 9.99999796e-05, 9.99999811e-05, 9.99999825e-05,
       9.99999837e-05, 9.99999849e-05, 9.99999860e-05, 9.99999869e-05,
       9.99999878e-05, 9.99999886e-05, 9.99999895e-05, 9.99999902e-05,
       9.99999908e-05, 9.99999915e-05, 9.99999920e-05, 9.99999926e-05,
       9.99999930e-05, 9.99999935e-05, 9.99999939e-05, 9.99999943e-05,
       9.99999947e-05, 9.99999950e-05, 9.99999953e-05, 9.99999957e-05,
      

In [8]:
# now check widths
kb = 8.3e-8
b = (kb * Q[1:] * (S ** (7/6))) / (D**(3/2))

In [9]:
b_list = list(b)
print(b_list)

[4.1419930785569425, 5.428379413364228, 6.6476100470567285, 7.890389487708792, 9.149184036208657, 10.414361846974186, 11.678609482807174, 12.936575639561857, 14.184308934886316, 15.418852563340204, 16.637972254610684, 17.83997209028892, 19.02356655939461, 20.187789162534067, 21.331925389037018, 22.45546240836372, 23.558050526837125, 24.63947313449985, 25.69962290536071, 26.738482699309557, 27.75611006714274, 28.752624530868918, 29.728197085522503, 30.683041435264954, 31.617406649600024, 32.53157097222093, 33.42583655997474, 34.300525023691584, 35.155973607607194, 35.992531917470735, 36.810559109871434, 37.61042148800879, 38.39249039235975, 39.157140437018114, 39.90474794213231, 40.63568960515666, 41.35034132937801, 42.049077272535456, 42.732268939280864, 43.400284499886745, 44.05348813483651, 44.69223950735895, 45.31689334064123, 45.92779898104016, 46.52530014037589, 47.10973461958547, 47.68143407287688, 48.24072387111511, 48.787922950714666, 49.32334374906129, 49.84729208698091, 50.36

In [10]:
# now convert discharge from m^3/yr to m^3/s
Q_pers = Q / 3.15e7

In [11]:
# ok really going out on a limb here...
depth = (0.09801 * D) / S

In [12]:
depth_list = list(depth)
print(depth_list)

[0.004445135487090938, 0.011567931703508564, 0.01948506540619229, 0.02754754654814288, 0.035572457720120694, 0.04351496196543352, 0.05137530502495247, 0.05916813345265465, 0.0669116517487259, 0.07462373556209527, 0.08232063762841502, 0.0900166865891105, 0.09772435878036034, 0.10545447501919546, 0.11321642293245868, 0.12101836660665315, 0.12886743097020353, 0.1367698588467032, 0.1447311427502909, 0.152756134757587, 0.1608491378959004, 0.16901398232987658, 0.17725408886172891, 0.18557252220407014, 0.19397203574954594, 0.20245510936538283, 0.2110239815260567, 0.21968067659544877, 0.22842702823933492, 0.23726469954775617, 0.24619520041534784, 0.25521990253161675, 0.2643400526518766, 0.2735567838795946, 0.28287112583696644, 0.29228401348867405, 0.3017962951062899, 0.3114087389818875, 0.3211220399753403, 0.3309368247516496, 0.3408536569537672, 0.35087304166250655, 0.36099542926404943, 0.37122121951601994, 0.38155076450021785, 0.39198437177616524, 0.40252230734617506, 0.4131647979942858, 0.42

In [13]:
Q_pers_short = Q_pers[1:]
print(Q_pers_short)

[1.05820106e-02 4.23280423e-02 9.52380952e-02 1.69312169e-01
 2.64550265e-01 3.80952381e-01 5.18518519e-01 6.77248677e-01
 8.57142857e-01 1.05820106e+00 1.28042328e+00 1.52380952e+00
 1.78835979e+00 2.07407407e+00 2.38095238e+00 2.70899471e+00
 3.05820106e+00 3.42857143e+00 3.82010582e+00 4.23280423e+00
 4.66666667e+00 5.12169312e+00 5.59788360e+00 6.09523810e+00
 6.61375661e+00 7.15343915e+00 7.71428571e+00 8.29629630e+00
 8.89947090e+00 9.52380952e+00 1.01693122e+01 1.08359788e+01
 1.15238095e+01 1.22328042e+01 1.29629630e+01 1.37142857e+01
 1.44867725e+01 1.52804233e+01 1.60952381e+01 1.69312169e+01
 1.77883598e+01 1.86666667e+01 1.95661376e+01 2.04867725e+01
 2.14285714e+01 2.23915344e+01 2.33756614e+01 2.43809524e+01
 2.54074074e+01 2.64550265e+01 2.75238095e+01 2.86137566e+01
 2.97248677e+01 3.08571429e+01 3.20105820e+01 3.31851852e+01
 3.43809524e+01 3.55978836e+01 3.68359788e+01 3.80952381e+01
 3.93756614e+01 4.06772487e+01 4.20000000e+01 4.33439153e+01
 4.47089947e+01 4.609523

In [17]:
Eb

array([0.00866379, 0.01079323, 0.0119543 , 0.01278647, 0.0134456 ,
       0.01399622, 0.01447176, 0.01489193, 0.01526943, 0.01561291,
       0.01592858, 0.01622105, 0.01649383, 0.01674967, 0.01699077,
       0.01721892, 0.01743558, 0.01764198, 0.01783915, 0.01802796,
       0.01820919, 0.01838347, 0.01855138, 0.01871343, 0.01887005,
       0.01902164, 0.01916854, 0.01931107, 0.01944952, 0.01958413,
       0.01971514, 0.01984276, 0.01996717, 0.02008856, 0.02020709,
       0.0203229 , 0.02043612, 0.02054689, 0.02065533, 0.02076153,
       0.0208656 , 0.02096764, 0.02106773, 0.02116595, 0.02126238,
       0.0213571 , 0.02145016, 0.02154164, 0.02163158, 0.02172006,
       0.02180712, 0.02189281, 0.02197717, 0.02206027, 0.02214212,
       0.02222279, 0.0223023 , 0.02238069, 0.022458  , 0.02253426,
       0.02260951, 0.02268376, 0.02275705, 0.02282941, 0.02290087,
       0.02297144, 0.02304116, 0.02311004, 0.02317812, 0.0232454 ,
       0.02331191, 0.02337767, 0.0234427 , 0.02350701, 0.02357

In [16]:
Eh

array([ 2.48049977e-13, -6.89160507e-13, -7.30577603e-13, -6.94607953e-13,
       -6.43510731e-13, -5.94302340e-13, -5.47860636e-13, -5.07031174e-13,
       -4.68234484e-13, -4.29882359e-13, -4.08730506e-13, -3.74480073e-13,
       -3.59874022e-13, -3.29950162e-13, -3.10648486e-13, -2.88485812e-13,
       -2.76710019e-13, -2.47898536e-13, -2.64831862e-13, -2.35707362e-13,
       -2.04572442e-13, -1.98413610e-13, -2.02900743e-13, -1.82226185e-13,
       -1.85190614e-13, -1.97902325e-13, -1.12753219e-13, -1.60831376e-13,
       -1.78193209e-13, -1.13145955e-13, -1.56982821e-13, -9.52507527e-14,
       -1.18026112e-13, -1.34065322e-13, -1.27626403e-13, -8.22379978e-14,
       -9.08053394e-14, -1.10237433e-13, -6.82038324e-14, -9.98515604e-14,
       -9.71369747e-14, -7.33620412e-14, -8.73028483e-14, -5.18649650e-14,
       -7.98536218e-14, -3.76565851e-14, -7.49303677e-14, -6.85780510e-14,
       -5.79152642e-14, -6.22389608e-14, -5.91661463e-14, -4.91269126e-14,
       -3.60384644e-14, -